In [1]:
import os
from langchain import hub
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
import chromadb
import chromadb.utils.embedding_functions as embedding_functions
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [2]:
load_dotenv()

True

In [3]:
os.environ["LANGCHAIN_TRACING_V2"] = os.getenv("LANGCHAIN_TRACING_V2")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

In [4]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [5]:
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=os.getenv("OPENAI_API_KEY"),
    model_name="text-embedding-3-small"
)

In [6]:
chroma_client = chromadb.HttpClient(host='localhost', port=8000)

In [7]:
collection = chroma_client.get_or_create_collection('blog', embedding_function=openai_ef)

In [8]:
vectorstore = Chroma(client=chroma_client, collection_name='blog', embedding_function=OpenAIEmbeddings())

In [9]:
retriever = vectorstore.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold":0.5})

In [10]:
prompt = hub.pull("rlm/rag-prompt")

In [11]:
example_message = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

In [12]:
print(example_message[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: filler question 
Context: filler context 
Answer:


In [13]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs , "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
for chunk in rag_chain.stream("What is Rater Agreement?"):
    print(chunk, end="", flush=True)

d:\Programacion\langchain\venv\Lib\site-packages\langchain_core\vectorstores\base.py:784: UserWarning: Relevance scores must be between 0 and 1, got [(Document(metadata={'id': 36, 'metadata': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'page_content': 'inquired about current trends in anticancer drug discovery;\nselected a target;\nrequested a scaffold targeting these compounds;\nOnce the compound was identified, the model attempted its synthesis.', 'tokens': 'inquired about current trends in anticancer drug discovery selected target requested scaffold targeting these compounds once the compound was identified the model attempted its synthesis'}, page_content='inquired about current trends in anticancer drug discovery selected target requested scaffold targeting these compounds once the compound was identified the model attempted its synthesis'), -0.3198669264430378), (Document(metadata={'id': 46, 'metadata': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'page_con

Rater agreement is a statistical measure that assesses the consistency of judgments made by two or more raters. It quantifies the level of agreement between raters on a particular task, such as evaluating essays or classifying images.  Rater agreement is important for ensuring the reliability and validity of research findings. 


#### Crear una Chain personalizada

In [14]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [17]:
system_prompt = (
    'Usted es un asistente para tareas de respuesta a preguntas. '
    'Si el texto de entrada esta en un idioma diferente al español, traduzca el texto al español.'
    'Utilice las siguientes piezas de contexto recuperado para responder '
    'la pregunta. Si no sabes la respuesta, di que '
    'No lo sé. Utilice tres oraciones como máximo y mantenga el '
    'respuesta concisa.'
    '\n\n'
    '{context}'
)

In [18]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [19]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain_2 = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain_2.invoke({"input": "What is Task Decomposition?"})
print(response["answer"])

d:\Programacion\langchain\venv\Lib\site-packages\langchain_core\vectorstores\base.py:784: UserWarning: Relevance scores must be between 0 and 1, got [(Document(metadata={'id': 19, 'metadata': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'page_content': 'LSH (Locality-Sensitive Hashing): It introduces a hashing function such that similar input items are mapped to the same buckets with high probability, where the number of buckets is much smaller than the number of inputs.\nANNOY (Approximate Nearest Neighbors Oh Yeah): The core data structure are random projection trees, a set of binary trees where each non-leaf node represents a hyperplane splitting the input space into half and each leaf stores one data point. Trees are built independently and at random, so to some extent, it mimics a hashing function. ANNOY search happens in all the trees to iteratively search through the half that is closest to the query and then aggregates the results. The idea is quite related to KD tre

La descomposición de tareas es el proceso de dividir una tarea grande y compleja en tareas más pequeñas y manejables. Esto ayuda a simplificar el trabajo, hacerlo más fácil de entender y administrar, y facilita la asignación de responsabilidades a diferentes personas o equipos. La descomposición de tareas es una técnica común utilizada en la gestión de proyectos, la ingeniería y otras áreas donde se necesitan tareas complejas. 

